In [2]:
import os
import boto3
import io
import sagemaker

import pandas as pd
import numpy as np

from sagemaker import KMeans

In [3]:
role = sagemaker.get_execution_role()
s3_client = boto3.client("s3")
s3Bucket = "sagemaker-studio-p0xlvix7lda"

In [4]:
source_data = "s3://sagemaker-studio-p0xlvix7lda/Gold daily processed final.csv"
response = s3_client.get_object(Bucket=s3Bucket, Key="Gold daily processed final.csv")
response_body = response["Body"].read()
table = pd.read_csv(io.BytesIO(response_body), header=0, delimiter=",", low_memory=False, parse_dates=['Date'], index_col=0)
table.head()

,Gold Price,S&P,Gold Return,Gold Return 2 day back 1,Gold Return 4 day back 3,Gold Return 8 day back 7,Var 16 day,Normalised Gold Return,Normalised Gold Return 2 day back 1,Normalised Gold Return 4 day back 3,...,S&P Return 8 day back 7,Var 16 day.1,Normalised S&P Return,Normalised S&P Return 2 day back 1,Normalised S&P Return 4 day back 3,Normalised S&P Return 8 day back 7,Tomorrow Gold,Tomorrow S&P,Tomorrow Gold Return,Tomorrow S&P Return
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-26,559.7,1273.83,-0.004447,0.006985,0.008854,0.036524,0.013745,-0.323525,0.359369,0.322096,...,0.007515,0.006625,1.092031,0.072626,-1.129939,0.401029,558.7,1283.72,-0.001787,0.007764
2006-01-27,558.7,1283.72,-0.001787,0.003586,0.025749,0.033061,0.013338,-0.133952,0.190116,0.965259,...,0.003494,0.006686,1.161302,0.581902,-0.647845,0.184792,565.8,1285.19,0.012708,0.001145
2006-01-30,565.8,1285.19,0.012708,-0.006226,0.006805,0.034649,0.012713,0.999577,-0.346257,0.267636,...,-0.000319,0.006686,0.171274,1.592265,-1.184882,-0.016867,570.8,1280.08,0.008837,-0.003976
2006-01-31,570.8,1280.08,0.008837,0.010899,0.011201,0.008013,0.011519,0.767178,0.669035,0.486221,...,-0.022214,0.006339,-0.627284,0.994861,0.771635,-1.239087,569.4,1282.46,-0.002453,0.001859
2006-02-01,569.4,1282.46,-0.002453,0.021657,0.000716,0.025721,0.010985,-0.223273,1.394069,0.032610,...,-0.020059,0.006281,0.296037,-0.319243,1.253555,-1.129202,572.5,1270.84,0.005444,-0.009061


In [5]:
trainTableNew = table.loc[:"2017-12-26",:]
testTableNew  = table.loc["2017-12-27":,:]

In [6]:
kMeansTable = table[['Normalised Gold Return', 'Normalised Gold Return 2 day back 1', 'Normalised Gold Return 4 day back 3', 'Normalised Gold Return 8 day back 7', 'Var 16 day', 'Normalised S&P Return', 'Normalised S&P Return 2 day back 1', 'Normalised S&P Return 4 day back 3', 'Normalised S&P Return 8 day back 7', 'Var 16 day.1']]
trainKMeansTableNew = kMeansTable.iloc[:3000,:]
testKMeansTableNew = kMeansTable.iloc[3000:,:]

In [7]:
trainKMeansTableNew.shape

(3000, 10)

In [8]:
testKMeansTableNew.shape

(992, 10)

In [9]:
num_clusters = 6
from sagemaker import KMeans

kmeans = KMeans(
    role=role,
    instance_count=1,
    instance_type="ml.c4.xlarge",
    output_path="s3://" + s3Bucket + "/groups/",
    k=num_clusters,
)


In [10]:
kmeans.fit(kmeans.record_set(trainKMeansTableNew.values.astype("float32")))

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-12-26 21:06:42 Starting - Starting the training job...
2021-12-26 21:07:06 Starting - Launching requested ML instancesProfilerReport-1640552801: InProgress
...
2021-12-26 21:07:41 Starting - Preparing the instances for training...............
2021-12-26 21:10:07 Downloading - Downloading input data...
2021-12-26 21:10:41 Training - Training image download completed. Training in progress.
2021-12-26 21:10:41 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[12/26/2021 21:10:36 INFO 140113885333312] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'init_method': 'random', 'mini_batch_size': '5000', 'epochs': '1', 'extra_center_factor': 'auto', 'local_lloyd_max_iter': '300', 'local_lloyd_tol': '0.0001', 'local_lloyd_init_method': 'kmeans++', 'local_lloyd_num_trials': 'auto', 'half_life_time_size': '0', 'eval_metrics': '["msd

In [11]:
kmeans_predictor = kmeans.deploy(initial_instance_count=1, instance_type="ml.t2.medium")

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


--------------!

In [12]:
trainResult = kmeans_predictor.predict(trainKMeansTableNew.values.astype("float32"))

In [13]:
testResult = kmeans_predictor.predict(testKMeansTableNew.values.astype("float32"))

In [14]:
kmeans_predictor.delete_endpoint()

In [15]:
trainResultList = [int(i.label["closest_cluster"].float32_tensor.values[0]) for i in trainResult]
trainTableNew['Cluster'] = trainResultList
trainTableNew.head()
trainTableNew.to_csv("clusteredTrain.csv")
[trainTableNew[trainTableNew['Cluster']==i].to_csv("train" + str(i) + ".csv") for i in np.unique(trainTableNew['Cluster'].values.ravel())]
trainTableClustered = [trainTableNew[trainTableNew['Cluster']==i] for i in np.unique(trainTableNew['Cluster'].values.ravel())]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
testResultList = [int(i.label["closest_cluster"].float32_tensor.values[0]) for i in testResult]
testTableNew['Cluster'] = testResultList
testTableNew.head()
testTableNew.to_csv("clusteredTest.csv")
[testTableNew[testTableNew['Cluster']==i].to_csv("test" + str(i) + ".csv") for i in np.unique(testTableNew['Cluster'].values.ravel())]
testTableClustered = [testTableNew[testTableNew['Cluster']==i] for i in np.unique(testTableNew['Cluster'].values.ravel())]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
trainTableClustered[0].head()

,Gold Price,S&P,Gold Return,Gold Return 2 day back 1,Gold Return 4 day back 3,Gold Return 8 day back 7,Var 16 day,Normalised Gold Return,Normalised Gold Return 2 day back 1,Normalised Gold Return 4 day back 3,...,Var 16 day.1,Normalised S&P Return,Normalised S&P Return 2 day back 1,Normalised S&P Return 4 day back 3,Normalised S&P Return 8 day back 7,Tomorrow Gold,Tomorrow S&P,Tomorrow Gold Return,Tomorrow S&P Return,Cluster
Date,,,,,,,,,,,,,,,,,,,,,
2006-02-03,567.4,1264.03,-0.008908,0.002978,0.015297,0.025351,0.010917,-0.816004,0.192907,0.700608,...,0.006626,-0.808743,-0.770325,0.918891,-0.887065,570.2,1265.02,0.004935,0.000783,0
2006-02-06,570.2,1265.02,0.004935,-0.003512,0.017331,0.006474,0.010906,0.452490,-0.227739,0.794561,...,0.006517,0.120171,-1.559146,0.519744,-0.580551,551.0,1254.78,-0.033672,-0.008095,0
2006-02-07,551.0,1254.78,-0.033672,-0.004017,0.024700,0.009577,0.013692,-2.459206,-0.207471,0.901968,...,0.006718,-1.204875,-0.482011,-0.746715,0.028301,550.1,1265.65,-0.001633,0.008663,0
2006-02-08,550.1,1265.65,-0.001633,-0.028904,0.002828,0.040647,0.013647,-0.119685,-1.497569,0.103603,...,0.007160,1.209889,-0.722691,-1.149746,0.280523,564.5,1263.78,0.026177,-0.001478,0
2006-02-22,553.9,1292.67,0.000000,0.014469,-0.007634,-0.036090,0.015552,0.000000,0.657840,-0.245414,...,0.005915,1.270293,-0.588764,1.493876,-0.611253,548.4,1287.79,-0.009930,-0.003775,0


In [ ]:
#
#model_key = "groups/" + kmeans.latest_training_job.name + "/output/model.tar.gz"
#boto3.resource("s3").Bucket(bucket).download_file(model_key, "model.tar.gz")
#os.system("tar -zxvf model.tar.gz")
#Kmeans_model_params = mx.ndarray.load("model_algo-1")
#cluster_centroids = pd.DataFrame(Kmeans_model_params[0].asnumpy())
#cluster_centroids.columns = counties_transformed.columns
#